In [3]:
pip install transformers==4.3.2

     |████████████████████████████████| 1.8MB 7.5MB/s 
     |████████████████████████████████| 890kB 32.2MB/s 
     |████████████████████████████████| 3.2MB 43.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=8c2319e183957335a357e7cac3fcc40ad4a30c3a55020503127bb076e05aca91
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
pip install fastapi nest-asyncio pyngrok uvicorn python-multipart

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 747kB 9.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 10.1MB 11.9MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.4-cp37-none-any.whl size=18971 sha256=2ad2e6b5bc4c6ce923d2ec4fb36e8b4674caf11cff818a1fd6f337644ce9f78d
  Stored in directory: /root/.cache/pip/wheels/8a/82/b1/cecfba4ff6e2f05777a5a4a65b46c1114842453d5a0e61bdd4
  Created wheel for python-multipart: filename=python_multipart-0.0.5-cp37-none-any.whl size=31671 sha256=29cd1dea7dd30a92958dadb428a1442ba93b8ac2d22855529de7023b47551446
  Stored in directory: /root/.cache/pip/wheels/f0/e6/66/14a866a3cbd6a0cabfbef91f7edf40aa03595ef6c88d6d1be4
Successfully built pyngrok python-multipart


In [31]:
labels = ["admiration", "amusement", "anger", "annoyance", "approval", "caring", 
          "confusion", "curiosity", "desire", "disappointment", "disapproval",
          "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief",
          "joy", "love", "nervousness", "optimism", "pride", "realization",
          "relief", "remorse", "sadness", "surprise", "neutral"]
index_to_labels = {index: label for index, label in enumerate(labels)}

In [13]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
import numpy as np
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

In [50]:
# making response body
class Prediction(BaseModel):
  data: str


app = FastAPI()

@app.get('/')
async def home():
  return "please go to /docs"

# path used for predicting after uploading image file
@app.post('/predict/')
async def predict_out(data):
  model_name = 'distilbert-base-uncased'
  config = AutoConfig.from_pretrained(model_name, num_labels=27)
  model = AutoModelForSequenceClassification.from_config(config=config)
  tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
  data = data
  #y = data['data']
  prepared_input = tokenizer.prepare_seq2seq_batch([data], return_tensors='pt')
  model = model.to('cpu')
  model.eval()
  model_output = model(**prepared_input)
  prediction = np.argmax(model_output.logits[0].detach().numpy())
  text = index_to_labels[prediction]
 
  return {
      'predict_class': text
 
  }
  

In [51]:
# using ngrok on google colab to expose url so it can be accessed from browser on my PC
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://25cc83f68d5b.ngrok.io


INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     45.116.232.62:0 - "GET / HTTP/1.1" 200 OK
INFO:     45.116.232.62:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     45.116.232.62:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     45.116.232.62:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     45.116.232.62:0 - "POST /predict/?data=Happy%20Birthday HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [58]
